# Finjustering af Open AI-modeller

Denne notesbog er baseret på den aktuelle vejledning i [Finjustering](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) dokumentationen fra Open AI.

Finjustering forbedrer ydeevnen af grundmodeller til din applikation ved at genuddanne den med yderligere data og kontekst, der er relevant for den specifikke brugssag eller situation. Bemærk, at prompt engineering-teknikker som _few shot learning_ og _retrieval augmented generation_ giver dig mulighed for at forbedre standardprompten med relevante data for at øge kvaliteten. Disse tilgange er dog begrænset af den maksimale token-vinduesstørrelse for den målrettede grundmodel.

Med finjustering genuddanner vi effektivt selve modellen med de nødvendige data (hvilket giver os mulighed for at bruge mange flere eksempler, end der kan være i det maksimale token-vindue) - og implementerer en _tilpasset_ version af modellen, som ikke længere behøver at få eksempler leveret ved inferenstidspunktet. Dette forbedrer ikke kun effektiviteten af vores promptdesign (vi har mere fleksibilitet i brugen af token-vinduet til andre ting), men forbedrer potentielt også vores omkostninger (ved at reducere antallet af tokens, vi skal sende til modellen ved inferenstidspunktet).

Finjustering har 4 trin:
1. Forbered træningsdataene og upload dem.
1. Kør træningsjobbet for at få en finjusteret model.
1. Evaluer den finjusterede model og iterer for kvalitet.
1. Implementer den finjusterede model til inferens, når du er tilfreds.

Bemærk, at ikke alle grundmodeller understøtter finjustering - [tjek OpenAI-dokumentationen](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) for den seneste information. Du kan også finjustere en tidligere finjusteret model. I denne vejledning bruger vi `gpt-35-turbo` som vores målrettede grundmodel til finjustering.

---


### Trin 1.1: Forbered dit datasæt

Lad os bygge en chatbot, der hjælper dig med at forstå det periodiske system ved at besvare spørgsmål om et grundstof med et limerick. I _denne_ simple vejledning vil vi blot oprette et datasæt til at træne modellen med nogle få eksempler på svar, der viser det forventede format for dataene. I en virkelig brugssituation ville du skulle oprette et datasæt med mange flere eksempler. Du kan også muligvis bruge et åbent datasæt (til dit anvendelsesområde), hvis et sådant findes, og omformatere det til brug i finjustering.

Da vi fokuserer på `gpt-35-turbo` og søger et enkelt-svar (chat completion), kan vi oprette eksempler ved hjælp af [dette foreslåede format](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst), som afspejler OpenAI's krav til chat completion. Hvis du forventer indhold med flere samtalerunder, ville du bruge [multi-turn eksempel formatet](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst), som inkluderer en `weight`-parameter for at angive, hvilke beskeder der skal bruges (eller ikke) i finjusteringsprocessen.

Vi vil bruge det simplere enkelt-svar format til vores vejledning her. Dataene er i [jsonl-format](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) med 1 post per linje, hver repræsenteret som et JSON-formateret objekt. Uddraget nedenfor viser 2 poster som eksempel - se [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) for det fulde eksempelsæt (10 eksempler), vi bruger til vores finjusteringsvejledning. **Bemærk:** Hver post _skal_ defineres på en enkelt linje (ikke opdelt over flere linjer som typisk i en formateret JSON-fil)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

I en virkelig brugssituation vil du have brug for et meget større eksempelsæt for gode resultater - afvejningen vil være mellem kvaliteten af svarene og tiden/omkostningerne til finjustering. Vi bruger et lille sæt, så vi hurtigt kan gennemføre finjusteringen for at illustrere processen. Se [dette OpenAI Cookbook-eksempel](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) for en mere kompleks finjusteringsvejledning.


---

### Trin 1.2 Upload dit datasæt

Upload dataene ved hjælp af Files API [som beskrevet her](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Bemærk, at for at køre denne kode, skal du først have udført følgende trin:
 - Installeret `openai` Python-pakken (sørg for at bruge en version >=0.28.0 for de nyeste funktioner)
 - Sat `OPENAI_API_KEY` miljøvariablen til din OpenAI API-nøgle
For at lære mere, se [Opsætningsvejledningen](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) leveret til kurset.

Kør nu koden for at oprette en fil til upload fra din lokale JSONL-fil.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Trin 2.1: Opret finjusteringsjobbet med SDK'en


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Trin 2.2: Tjek status for jobbet

Her er nogle ting, du kan gøre med `client.fine_tuning.jobs` API'et:
- `client.fine_tuning.jobs.list(limit=<n>)` - List de sidste n fine-tuning jobs
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Få detaljer om et specifikt fine-tuning job
- `client.fine_tuning.jobs.cancel(<job_id>)` - Annuller et fine-tuning job
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - List op til n begivenheder fra jobbet
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Det første trin i processen er _at validere træningsfilen_ for at sikre, at data er i det rigtige format.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Trin 2.3: Spor begivenheder for at overvåge fremskridt


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Trin 2.4: Se status i OpenAI Dashboard


Du kan også se status ved at besøge OpenAI-webstedet og udforske afsnittet _Fine-tuning_ på platformen. Dette vil vise dig status for det aktuelle job og også lade dig spore historikken for tidligere jobkørsler. På dette skærmbillede kan du se, at den tidligere kørsel mislykkedes, og at den anden kørsel lykkedes. Til kontekst skete dette, da den første kørsel brugte en JSON-fil med forkert formaterede poster – når det blev rettet, blev den anden kørsel fuldført med succes og gjorde modellen tilgængelig til brug.

![Fine-tuning job status](../../../../../translated_images/da/fine-tuned-model-status.563271727bf7bfba.png)


Du kan også se statusmeddelelserne og målingerne ved at rulle længere ned i det visuelle dashboard som vist:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/da/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/da/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Trin 3.1: Hent ID & Test Finjusteret Model i Kode


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Trin 3.2: Indlæs & test finjusteret model i Playground

Du kan nu teste den finjusterede model på to måder. Først kan du besøge Playground og bruge Models-rullemenuen til at vælge din nyfinjusterede model fra de listede muligheder. Den anden mulighed er at bruge "Playground"-valget vist i Fine-tuning-panelet (se skærmbilledet ovenfor), som åbner følgende _sammenlignende_ visning, der viser grundmodellen og den finjusterede model side om side til hurtig evaluering.

![Fine-tuning job status](../../../../../translated_images/da/fine-tuned-playground-compare.56e06f0ad8922016.png)

Udfyld blot systemkonteksten, der blev brugt i dine træningsdata, og angiv dit testspørgsmål. Du vil bemærke, at begge sider opdateres med den identiske kontekst og spørgsmål. Kør sammenligningen, og du vil se forskellen i output mellem dem. _Bemærk hvordan den finjusterede model gengiver svaret i det format, du har angivet i dine eksempler, mens grundmodellen blot følger systemprompten_.

![Fine-tuning job status](../../../../../translated_images/da/fine-tuned-playground-launch.5a26495c983c6350.png)

Du vil bemærke, at sammenligningen også viser token-tællinger for hver model og den tid, der er brugt til inferens. **Dette specifikke eksempel er et simpelt et, der er ment til at vise processen, men som ikke afspejler et virkeligt datasæt eller scenarie**. Du kan bemærke, at begge prøver viser det samme antal tokens (systemkontekst og brugerprompt er identiske), hvor den finjusterede model tager længere tid til inferens (tilpasset model).

I virkelige scenarier vil du ikke bruge et legetøjseksempel som dette, men finjustere mod rigtige data (f.eks. produktkatalog til kundeservice), hvor kvaliteten af svaret vil være meget mere tydelig. I _den_ kontekst vil det kræve mere tilpasset prompt-engineering at opnå en tilsvarende svar-kvalitet med grundmodellen, hvilket vil øge tokenforbruget og potentielt den relaterede behandlingstid til inferens. _For at prøve dette, se fine-tuning-eksemplerne i OpenAI Cookbook for at komme i gang._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfraskrivelse**:
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, bedes du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det oprindelige dokument på dets modersmål bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der opstår som følge af brugen af denne oversættelse.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
